In [1]:

import sympy

from mlscorecheck.core import score_function_set

In [5]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')

eq = x**3*y - sympy.sqrt(x)*y**2 + 5

In [24]:
eq.has(sympy.Pow)

True

In [12]:
eq.args[2]

-sqrt(x)*y**2

In [6]:
eq.has(sympy.sqrt)

False

In [56]:
scores = score_function_set()

In [57]:
score0 = 'ppv'
score1 = 'ji'

score0_fun = scores[score0]
score1_fun = scores[score1]

In [58]:
tp = sympy.Symbol('tp')
tn = sympy.Symbol('tn')
fp = sympy.Symbol('fp')
fn = sympy.Symbol('fn')
p = sympy.Symbol('p')
n = sympy.Symbol('n')

symbols = {'tp': tp,
            'tn': tn,
            'fp': fp,
            'fn': fn,
            'p': p,
            'n': n}

score0_sym = sympy.Symbol(score0)
score1_sym = sympy.Symbol(score1)

In [59]:
score0_args = score0_fun.__code__.co_varnames[:score0_fun.__code__.co_kwonlyargcount]
score1_args = score1_fun.__code__.co_varnames[:score1_fun.__code__.co_kwonlyargcount]

In [60]:
score0_args, score1_args

(('tp', 'fp'), ('tp', 'fp', 'p'))

In [61]:
score0_exp = score0_fun(**{key: val for key, val in symbols.items() if key in score0_args})
score1_exp = score1_fun(**{key: val for key, val in symbols.items() if key in score1_args})
score0_eq = score0_sym - score0_exp
score1_eq = score1_sym - score1_exp
n_eq = n - tn - fp
p_eq = p - tp - fn

In [62]:
score0_eq, score1_eq, n_eq, p_eq

(ppv - tp/(fp + tp), ji - tp/(fp + p), -fp + n - tn, -fn + p - tp)

In [63]:
score0_exp, score1_exp

(tp/(fp + tp), tp/(fp + p))

In [64]:
num0, denom0 = score0_exp.as_numer_denom()
num1, denom1 = score1_exp.as_numer_denom()

In [65]:
num0, denom0, num1, denom1

(tp, fp + tp, tp, fp + p)

In [66]:
score0_eq = sympy.simplify(score0_eq * denom0)
score1_eq = sympy.simplify(score1_eq * denom1)

In [67]:
score0_eq, score1_eq, n_eq, p_eq

(ppv*(fp + tp) - tp, ji*(fp + p) - tp, -fp + n - tn, -fn + p - tp)

In [68]:
gb = sympy.GroebnerBasis([score0_eq, score1_eq, n_eq, p_eq], tp, tn, fn, fp, order='grevlex')

In [69]:
gb

GroebnerBasis([-ji*p*ppv + tp*(ji*ppv - ji + ppv), -ji*n*ppv - ji*p*ppv + ji*n + ji*p - n*ppv + tn*(ji*ppv - ji + ppv), ji*p - p*ppv + fn*(ji*ppv - ji + ppv), ji*p*ppv - ji*p + fp*(ji*ppv - ji + ppv)], tp, tn, fn, fp, domain='ZZ[p,n,ji,ppv]', order='grevlex')

In [70]:
gb[0], gb[1], gb[2], gb[3]

(-ji*p*ppv + tp*(ji*ppv - ji + ppv),
 -ji*n*ppv + ji*n - ji*p*ppv + ji*p - n*ppv + tn*(ji*ppv - ji + ppv),
 fn*(ji*ppv - ji + ppv) + ji*p - p*ppv,
 fp*(ji*ppv - ji + ppv) + ji*p*ppv - ji*p)

In [71]:
gb.is_zero_dimensional

True

In [79]:
sympy.solve(score1_sym*score0_sym - score1_sym + score0_sym)

[{ji: -ppv/(ppv - 1)}]

In [77]:
print(sympy.Poly(gb[3], tp, tn, fp, fn).LM())

tp**0*tn**0*fp**1*fn**0


In [35]:
#ji*ppv - ji + ppv = 0
#ji*ppv = ji - ppv

In [36]:
sympy.solve(score1_exp * score0_exp - score1_exp + score0_exp)

[{p: 0}, {tp: 0}]

In [39]:
subst = {score0_sym: 0}
score0_eq_new = score0_eq.subs(subst)
score1_eq_new = score1_eq.subs(subst)
n_eq_new = n_eq.subs(subst)
p_eq_new = p_eq.subs(subst)

In [42]:
score0_eq_new, score1_eq_new, n_eq_new, p_eq_new

(-tp, ji*(fp + p) - tp, -fp + n - tn, -fn + p - tp)

In [46]:
sympy.GroebnerBasis([score0_eq_new, score1_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='grevlex')

GroebnerBasis([tp, -n - p + tn, fn - p, fp + p], tp, tn, fn, fp, domain='ZZ[p,n,ji]', order='grevlex')

In [49]:
gb[0].subs({score0_sym: 0}) / score1_sym

-tp

In [51]:
sympy.simplify(gb[1].subs({score0_sym: 0}) / score1_sym)

n + p - tn

In [53]:
sympy.simplify(gb[2].subs({score0_sym: 0}) / score1_sym)

-fn + p

In [54]:
sympy.simplify(gb[3].subs({score0_sym: 0}) / score1_sym)

-fp - p

In [ ]:
sympy.GroebnerBasis([spec_eq_new, npv_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='lex').is_zero_dimensional

False

In [ ]:
spec_eq_new, npv_eq_new, n_eq_new, p_eq_new

((n + p)*(n*spec - tn)/n, -tn, -fp + n - tn, -fn + p - tp)

In [ ]:
sympy.GroebnerBasis([acc_eq_new, npv_eq_new, n_eq_new, p_eq_new], tp, tn, fn, fp, order='lex')

GroebnerBasis([-acc*n - acc*p + tp, tn, acc*n + acc*p + fn - p, fp - n], tp, tn, fn, fp, domain='ZZ[p,n,acc]', order='lex')

In [ ]:
sympy.solve_poly_system([acc_eq, npv_eq, n_eq, p_eq], tp, tn, fp, fn)

[((acc*n*npv - acc*n + acc*npv*p - acc*p + npv*p)/(2*npv - 1),
  npv*(acc*n + acc*p - p)/(2*npv - 1),
  -(acc*n*npv + acc*npv*p - 2*n*npv + n - npv*p)/(2*npv - 1),
  -(npv - 1)*(acc*n + acc*p - p)/(2*npv - 1))]